# Natural Language Processing + Decision Science

👩🏻‍🏫 In this challenge, we will combine:
* 🗣 Natural Language Processing 
* 📊 Decision Science 

🎯 The goal is to understand the bad reviews of the products and the sellers on Olist

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",None)

# Machine Learning
from sklearn.pipeline import make_pipeline

# Language Processing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import string
import unidecode as unidecode

# Vectorizers and NLP Models
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

🕵🏻‍♂️ Imagine that the CEO of Olist [Tiago Dalvi](https://www.linkedin.com/in/tiagodalvi/) ask you to read and understand the reviews.

- What did customers say about their order if they rated it 1? 2? 3?
- What are the most frequent bad reviews about...
    - the worst rated products ?
    - the wort sellers ?

## (0) Setup 🔨

First we'll load up the DataFrame which contains all the relevant information on Olist reviews!

In [3]:
df = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/olist/olist_reviews.csv")

In [4]:
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review
0,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,Recebi bem antes do prazo estipulado.
1,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,Parabéns lojas lannister adorei comprar pela ...
2,8670d52e15e00043ae7de4c01cc2fe06,174,4,b9bf720beb4ab3728760088589c62129,eletroportateis,recomendo aparelho eficiente. no site a marca ...
3,4b49719c8a200003f700d3d986ea1a19,45,4,9d6f15f95d01e79bd1349cc208361f09,beleza_saude,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,3948b09f7c818e2d86c9a546758b2335,56,5,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,"Super recomendo Vendedor confiável, produto ok..."


## (2) Text Cleaning

🧹 Create a function `cleaning(sentence)` and apply it to the reviews. **Beware NLTK has no Portuguese lemmatizer** (often but they do have a stemmer `nltk.stem.RSLPStemmer`. 

In [5]:
from nltk.stem import RSLPStemmer
nltk.download("rslp")
stemmer = RSLPStemmer()

import nltk

nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('rslp', quiet=True)

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\youne\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [6]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

stop_words = set(stopwords.words('portuguese')) ## defining stopwords

def cleaning(sentence):

    # Basic cleaning
    sentence = sentence.lower() ## lowercasing
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## removing numbers
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## removing punctuation
    # Advanced cleaning
    tokenized_sentence = word_tokenize(sentence) ## tokenizing

    tokenized_sentence = [w for w in tokenized_sentence
                                  if not w in stop_words] ## remove stopwords
    stemmed_sentence = [stemmer.stem(word)
              for word in tokenized_sentence] ## get word stems
    decoded_sentence = [unidecode.unidecode(w) for w in stemmed_sentence] ## remove accents

    cleaned_sentence = ' '.join(decoded_sentence) ## join back into a string

    return cleaned_sentence

In [7]:
df["full_review_cleaned"] = df['full_review'].apply(cleaning)
df["full_review_cleaned"].head()

0                           receb bem ant praz estipul
1    parabem loj lannist ador compr internet segur ...
2    recom aparelh efici sit marc aparelh impress d...
3                          pouc travandopel val ta boa
4       sup recom vend confi produt ok entreg ant praz
Name: full_review_cleaned, dtype: object

## (3) Analysis of bad reviews

### (3.1) Dataset with low review scores

😱 What is the proportion of reviews with a rating between 1 and 3 ? 

In [8]:
round(df["review_score"].value_counts(normalize = True),2)

review_score
5    0.57
4    0.16
1    0.13
3    0.09
2    0.05
Name: proportion, dtype: float64

🕵🏻‍♂️ Let's focus on these reviews...

In [9]:
threshold = 3
bad = df["review_score"] <= threshold
df = df[bad]
df.shape

(9658, 7)

In [10]:
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review,full_review_cleaned
14,e233e51d11511bf30e568c76360ace52,97,1,548df2c6e5f089574614894bca78acf5,eletronicos,recebi somente 1 controle Midea Split ESTILO....,receb soment control mide split estil falt con...
23,8b230a1373c6dc4bd867099fda1d7039,60,3,071251fe3b3493294536f03737a8a679,ferramentas_jardim,Eu comprei duas unidades e só recebi uma e ag...,compr dua unidad receb agor fac
24,cb2fc3e5711b5ae85e0491ee18af63ed,72,3,34e6d418f368f8079ae152bc178bc66a,beleza_saude,"Produto bom, porém o que veio para mim não co...",produt bom porem vei mim condiz fot anunci
25,60c714ed14cef913944a3147094a4742,36,1,9ac05114800f02bfaa783bd76842dbe2,moveis_decoracao,"Produto muito inferior, mal acabado.",produt inferi mal acab
28,0bd4dcc4f6c4621baf37f73495cad8c4,16,3,a11cd01ac67beef7e8bf09740d8a35c1,esporte_lazer,Entrega no prazo,entreg praz


### (3.2) Vectorizing

🔡 ➡️ 🔢 Vectorize your texts. 

* Make sure to take into accounts bigrams.
* Set a `max_df` to $0.75$ to remove too frequent words
* Spoiler alert: you will end up with $20k+$ words... let's keep only `max_features` = $5000$ for this challenge.

In [11]:
vectorizer = TfidfVectorizer(max_df = 0.75, max_features = 5000, ngram_range=(1,2))
vectorized_reviews = pd.DataFrame(vectorizer.fit_transform(df["full_review_cleaned"]).toarray(),
                                 columns = vectorizer.get_feature_names_out())

print(f" vectorized_reviews.shape = {vectorized_reviews.shape}")

 vectorized_reviews.shape = (9658, 5000)


### (3.3) LDA

🕵🏻‍♂️ Fit an LDA:
- Choose `n_components = 3`
- Show the Document Mixture of Topics  with *.transform()*
- Show the Topic Mixture with *.components_*

In [12]:
n_components = 3

lda = LatentDirichletAllocation(n_components = n_components)
lda.fit(vectorized_reviews)

,n_components,3
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


#### Document Mixture (of Topics)

In [13]:
document_mixture = lda.transform(vectorized_reviews)
document_mixture.shape

(9658, 3)

In [14]:
round(pd.DataFrame(document_mixture,
                   columns = [f"Topic {i+1}" for i in range(n_components)])
      ,2)

,Topic 1,Topic 2,Topic 3
0,0.08,0.23,0.69
1,0.08,0.84,0.08
2,0.83,0.08,0.09
3,0.47,0.10,0.42
4,0.13,0.13,0.74
...,...,...,...
9653,0.22,0.65,0.13
9654,0.17,0.70,0.12
9655,0.64,0.18,0.18
9656,0.11,0.79,0.10


👉 Let's report the most important topic for each review

In [15]:
df["most_important_topic"] = np.argmax(document_mixture, axis = 1)
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review,full_review_cleaned,most_important_topic
14,e233e51d11511bf30e568c76360ace52,97,1,548df2c6e5f089574614894bca78acf5,eletronicos,recebi somente 1 controle Midea Split ESTILO....,receb soment control mide split estil falt con...,2
23,8b230a1373c6dc4bd867099fda1d7039,60,3,071251fe3b3493294536f03737a8a679,ferramentas_jardim,Eu comprei duas unidades e só recebi uma e ag...,compr dua unidad receb agor fac,1
24,cb2fc3e5711b5ae85e0491ee18af63ed,72,3,34e6d418f368f8079ae152bc178bc66a,beleza_saude,"Produto bom, porém o que veio para mim não co...",produt bom porem vei mim condiz fot anunci,0
25,60c714ed14cef913944a3147094a4742,36,1,9ac05114800f02bfaa783bd76842dbe2,moveis_decoracao,"Produto muito inferior, mal acabado.",produt inferi mal acab,0
28,0bd4dcc4f6c4621baf37f73495cad8c4,16,3,a11cd01ac67beef7e8bf09740d8a35c1,esporte_lazer,Entrega no prazo,entreg praz,2


#### Topic Mixture (of Words)

In [16]:
topic_mixture = pd.DataFrame(lda.components_,
                             columns = vectorizer.get_feature_names_out())
topic_mixture.shape

(3, 5000)

In [17]:
topic_mixture

abaix  abaix expect    abajur      abert  abert produt   aborrec  \
0  0.358347      0.341984  0.447383   0.464486      0.355770  1.423406   
1  0.371819      0.334260  1.304445   0.398170      0.350790  1.541174   
2  4.154711      1.215793  0.535801  13.054928      1.393117  0.397121   

         abr  abr caix  abr cham  abr embal  abr fech  abr reclam     abril  \
0  11.873242  0.418930  0.335463   2.072869  0.361040    1.617789  1.737291   
1   7.481178  1.722785  1.880676   0.437102  0.334767    4.677665  0.481054   
2   5.126203  1.064234  0.337350   1.908578  1.759921    0.340198  0.337226   

     absurd  absurd entreg       acab  acab compr  acab deix  acab fic  \
0  7.587048       0.999168  23.115751    0.377966   0.341219  1.918270   
1  3.565062       1.120150   0.349015    0.634010   0.334168  0.335370   
2  2.232820       0.737946  11.541357    1.369197   2.713347  0.375303   

   acab produt  acab receb  acab ruim     aceit     acend     acert     acess  \
0     0.582689    2.150593   3.957038  3.249100  0.593450  0.337196  1.349466   
1     0.338068    0.597723   0.334646  1.865346  0.335123  1.795930  1.277843   
2     1.667491    0.371248   0.344208  2.725607  2.435606  0.537345  1.071936   

   acessori        ach  ach absurd   ach bem   ach car  ach cord  ach demor  \
0  0.759739  39.890626    1.423273  0.513140  1.234359  0.336373   0.338743   
1  2.769667   0.729130    0.337470  0.334874  0.337146  1.124634   0.335177   
2  1.432840  32.708742    0.346508  2.026311  0.728792  0.335379   5.352998   

    ach dev  ach entreg  ach estranh  ach fornec  ach fret   ach loj  \
0  4.724556    0.567177     0.995027    1.241996  0.391870  1.921256   
1  0.474757    0.337892     0.361212    0.333876  0.334889  0.338794   
2  0.435588    2.192213     1.209040    0.333868  2.609419  0.618749   

   ach mant  ach mater  ach melhor  ach pequen  ach pess  ach pouc  ach praz  \
0  2.123245   3.114218    0.353854    4.764942  2.026989  1.681526  0.340160   
1  0.334296   0.336669    0.334917    0.338281  0.339023  0.336929  0.334995   
2  0.336022   0.340690    2.078241    0.365361  0.349236  3.286432  3.252342   

   ach prec  ach produt  ach qual   ach vei      acid      acim     acion  \
0  0.335027    4.435411  0.336842  0.337713  0.347761  1.031953  0.735517   
1  0.334401    0.351752  0.334265  0.464593  0.340573  1.590749  1.126407   
2  2.610827    5.560066  2.255233  2.123072  1.813416  0.336210  1.729799   

        aco  acompanh  acompanh ped  acompanh produt  acondicion    acontec  \
0  1.023733  0.357240      0.343158         0.354803    2.663597  11.577408   
1  1.654461  4.090769      0.467125         0.748829    0.343452   7.717160   
2  0.368339  5.406149      2.222306         1.877424    0.494405   0.776632   

   acontec compr  acontec produt      acord  acord descr   acostum  \
0       1.478029        2.873798  11.030551     1.499908  0.503263   
1       0.373852        0.380753   0.367019     0.407227  0.342390   
2       0.347588        0.353420   0.413142     0.355318  2.249684   

   acostum compr   acredit     acril  acril fic      acus     adapt     adequ  \
0       0.381281  5.503226  0.407463   0.334780  0.775589  2.234712  0.838260   
1       0.347168  2.152083  3.550933   1.845891  2.565262  1.421126  0.502389   
2       1.562489  5.045994  1.627525   0.393322  0.356289  2.219321  4.919345   

       ader      ades  ades pared  ades pra    adiant       ador     adquir  \
0  0.346369  8.911397    1.698010  0.333854  1.128319   0.440857  12.675902   
1  0.335500  0.455006    0.583003  0.333755  1.745781   0.336085   2.555885   
2  2.209475  1.601232    0.339131  1.109304  0.354644  10.573157   0.721437   

   adquir consegu  adquir produt     adult      afet  afet ades    afinal  \
0        1.678162       3.310362  1.067521  0.336827   0.333854  0.347716   
1        0.338759       0.377499  1.122498  0.338511   0.333755  1.458931   
2        0.334081       0.334853  0.343180  2.357011   1.109304

#### Topics

🎁 We provided you with some functions:
* `topic_word` returns the top words with their weights for one topic
* `print_topics` prints the different topics found by the LDA with their topwords

In [18]:
def topic_word(vectorizer, model, topic, topwords, with_weights = True):
    topwords_indexes = topic.argsort()[:-topwords - 1:-1]
    if with_weights == True:
        topwords = [(vectorizer.get_feature_names_out()[i], round(topic[i],2)) for i in topwords_indexes]
    if with_weights == False:
        topwords = [vectorizer.get_feature_names_out()[i] for i in topwords_indexes]
    return topwords

In [19]:
def print_topics(vectorizer, model, topwords):
    for idx, topic in enumerate(model.components_):
        print("-"*20)
        print("Topic %d:" % (idx))
        print(topic_word(vectorizer, model, topic, topwords))


🕵🏻‍♂️ Print the topics with their topwords:

In [20]:
print_topics(vectorizer, lda, topwords = 5)

--------------------
Topic 0:
[('bom', np.float64(194.1)), ('produt', np.float64(117.6)), ('vei', np.float64(92.89)), ('compr', np.float64(80.86)), ('pess', np.float64(73.9))]
--------------------
Topic 1:
[('receb', np.float64(289.62)), ('produt', np.float64(198.11)), ('compr', np.float64(178.21)), ('entreg', np.float64(157.86)), ('receb produt', np.float64(127.1))]
--------------------
Topic 2:
[('produt', np.float64(206.6)), ('praz', np.float64(129.37)), ('entreg', np.float64(124.83)), ('nao', np.float64(104.78)), ('vei', np.float64(102.34))]


🇧🇷 A bit of Brazilian Portuguese words here:
- _cadeiras = chairs_
- _produto = product_
- _recomendo = recommend (não recomendo == not recommend)_
- _bom = good_
- _comprei = bought_
- _veio = came_
- _errado = wrong_
- _gostaria = I would like to..._
- _duas = two_
- _nao = not_
- _entregue = delivered_
- _pecas = part_
- _ainda = yet_
- _recebi = received_

👉 Show the top words associated to a topic

In [21]:
topic_word_mixture = [topic_word(vectorizer, lda, topic, topwords = 5, with_weights = False)
                      for topic in lda.components_]
topic_word_mixture

[['bom', 'produt', 'vei', 'compr', 'pess'],
 ['receb', 'produt', 'compr', 'entreg', 'receb produt'],
 ['produt', 'praz', 'entreg', 'nao', 'vei']]

In [22]:
df["most_important_words"] = df["most_important_topic"].apply(lambda i: topic_word_mixture[i])

In [23]:
df[["review_id",
        "review_score",
        "product_category_name",
        "full_review_cleaned",
        "most_important_topic",
        "most_important_words"]
      ].head()

,review_id,review_score,product_category_name,full_review_cleaned,most_important_topic,most_important_words
14,e233e51d11511bf30e568c76360ace52,1,eletronicos,receb soment control mide split estil falt con...,2,"[produt, praz, entreg, nao, vei]"
23,8b230a1373c6dc4bd867099fda1d7039,3,ferramentas_jardim,compr dua unidad receb agor fac,1,"[receb, produt, compr, entreg, receb produt]"
24,cb2fc3e5711b5ae85e0491ee18af63ed,3,beleza_saude,produt bom porem vei mim condiz fot anunci,0,"[bom, produt, vei, compr, pess]"
25,60c714ed14cef913944a3147094a4742,1,moveis_decoracao,produt inferi mal acab,0,"[bom, produt, vei, compr, pess]"
28,0bd4dcc4f6c4621baf37f73495cad8c4,3,esporte_lazer,entreg praz,2,"[produt, praz, entreg, nao, vei]"


## (3.4) Pipeline Tf-Idf and LDA

In [24]:
from sklearn import set_config
set_config("diagram")

🔨 Create a Pipeline that chains your previous Vectorizer and the LDA.

Fit it on the cleaned texts.

In [25]:
%%time
# Parameters
max_df = 0.75
max_features = 5000
ngram_range = (1,2)

# Pipeline Vectorizer + LDA
pipeline = make_pipeline(
    TfidfVectorizer(max_df = max_df,
                    max_features = max_features,
                    ngram_range = ngram_range),
    LatentDirichletAllocation(n_components = n_components)
)

# Fit the pipeline on the cleaned texts
pipeline.fit(df["full_review_cleaned"])

CPU times: total: 24.8 s
Wall time: 24.9 s


,steps,"[('tfidfvectorizer', ...), ('latentdirichletallocation', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


💡 If you try to access the components with `pipeline.components_`, it will NOT work because Pipeline doesn't have a `components_`. However, you can use `pipeline._final_estimator` to access the LDA. And from this, you can access the topics!

In [26]:
pipeline._final_estimator

,n_components,3
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


In [27]:
pipeline._final_estimator.components_

array([[2.29161282, 1.21009176, 0.58271171, ..., 0.48666695, 0.3604024 ,
        6.26112828],
       [1.20547118, 0.34746005, 1.36847849, ..., 1.70110816, 5.14045321,
        0.34738774],
       [1.38779268, 0.33448594, 0.33643818, ..., 1.31126144, 3.74547268,
        0.34990939]], shape=(3, 5000))

**Document Mixture** with the Pipeline:

In [28]:
# Transform the original cleaned texts with the pipeline
# Indeed, there is no need to get the vectorized texts first since it's done through the Pipeline
document_mixture = pipeline.transform(df["full_review_cleaned"])
document_mixture.shape

(9658, 3)

**Topic Mixture** with the Pipeline:

In [29]:
topic_mixture = pd.DataFrame(pipeline._final_estimator.components_)
topic_mixture.shape

(3, 5000)

In [30]:
topic_mixture

0         1         2         3         4         5          6     \
0  2.291613  1.210092  0.582712  3.140573  0.907178  1.595671   6.469189   
1  1.205471  0.347460  1.368478  8.821424  0.824833  1.418067  15.999364   
2  1.387793  0.334486  0.336438  1.955587  0.367668  0.347963   2.012070   

       7         8         9         10        11        12        13    \
0  0.834652  0.340166  1.439701  0.980063  0.343853  1.780283  1.531134   
1  2.009946  1.874237  2.597593  0.335824  5.848353  0.402305  3.220771   
2  0.361351  0.339085  0.381254  1.139841  0.443446  0.372983  8.633025   

       14         15        16        17        18        19        20    \
0  0.334555  33.689129  1.427957  2.718370  1.768102  1.918930  2.375170   
1  0.334004   0.443285  0.335356  0.334031  0.391915  0.334430  0.372350   
2  2.188704   0.873709  0.617861  0.336334  0.468925  0.334888  0.372044   

       21        22        23        24        25        26         27    \
0  3.967304  0.347891  2.665512  1.236402  1.345956  0.431439  51.090326   
1  0.334361  2.668731  0.341005  1.053909  1.947184  4.165397   1.186053   
2  0.334226  4.823432  0.357662  0.380160  0.406105  0.365410  21.052120   

       28        29        30        31        32        33        34    \
0  0.346484  2.146753  1.580983  0.357357  0.342680  0.355582  0.426342   
1  0.348130  0.353275  0.336756  0.333865  0.335924  0.340273  0.335850   
2  1.412637  0.374297  0.382559  1.105164  5.348314  4.939046  2.335091   

       35        36        37        38        39        40        41    \
0  1.183082  0.333981  0.602766  0.393721  2.125440  3.116800  0.591440   
1  1.012959  0.333752  0.337540  0.335154  0.334028  0.340504  0.574546   
2  0.369238  1.242007  2.395872  2.149923  0.334095  0.334273  1.601026   

       42        43        44        45        46        47        48    \
0  4.788112  1.987949  4.605972  0.356509  2.475498  9.590189  2.254947   
1  0.337486  0.367321  0.337718  0.339283  0.456610  0.339449  0.334260   
2  0.342986  0.359977  0.361197  3.231705  0.348146  0.417592  0.337133   

       49        50        51        52        53        54        55    \
0  1.342506  1.566214  0.797418  1.403342  1.565180  2.195755  0.334571   
1  0.353634  0.336507  1.816713  0.431468  0.924995  5.040489  2.353768   
2  1.229238  0.599028  0.344780  1.756913  0.556358  2.617914  0.344250   

       56        57        58        59        60        61        62    \
0  0.354722  1.579347  3.988506  1.337069  2.871177  1.994010  1.177985   
1  1.076759  0.347198  6.686937  0.342525  0.351568  3.341419  0.572267   
2  1.549575  1.574909  9.395757  0.519874  0.385225  6.475284  0.512201   

       63        64         65        66        67        68        69    \
0  0.822504  0.391253   1.292182  0.754035  0.334830  1.512553  0.560735   
1  0.757230  0.614152   0.913770  0.338250  0.334000  1.791915  1.047968   
2  1.515604  1.285533  10.495352  4.493637  1.905162  0.392672  4.266456   

       70        71        72        73        74        75         76    \
0  5.507491  2.222649  5.035661  0.404988  0.344008  0.461874   0.451012   
1  0.377804  0.334002  5.584745  1.877590  1.098841  2.064016   0.340094   
2  0.374699  0.334694  0.347230  0.337566  0.334064  0.702855  10.558993   

       77        78        79        80        81        82        83    \
0  5.994029  0.338899  0.365535  0.419433  1.723253  0.344008  0.350687   
1  3.517764  0.345691  0.353946  0.672832  0.942933  1.098841  1.630153   
2  6.441430  1.666412  3.303233  1.440934  0.366163  0.334064  0.486213   

       84        85        86        87        88         89        90    \
0  0.679990  0.338514  0.338918  0.342046  0.337177   1.939956  0.337343   
1  0.355584  0.337631  0.338333  0.336884  0.334130  49.765217  1.515892   
2  1.502115  8.463457  6.034885  5.281257  1.906754   6.233491  0.942766   

       91        92        93        94        95        96        97 

## (4) 🎁 Product Categories

### (4.1) Groupby product categories

📈 Group the dataset by `product_category_name` and inspect their performances

In [31]:
# Product categories by performance - look at the count, mean, median and std
product_categories = df.groupby(by = 'product_category_name').agg({
        'review_score': ["count", "mean", "median", "std"]
    })

# Removing products which were sold less than a certain times for the analysis
cutoff = 50
product_categories = product_categories[product_categories[("review_score", "count")] > cutoff]

# Sorting the product categories by performance
product_categories = product_categories.sort_values(by = [('review_score', 'mean'),
                                                          ('review_score', 'std')],
                                                    ascending = [False, True])
product_categories

review_score                           
                                         count      mean median       std
product_category_name                                                    
consoles_games                              92  1.956522    2.0  0.924787
fashion_bolsas_e_acessorios                155  1.948387    2.0  0.917319
malas_acessorios                            66  1.924242    2.0  0.949666
papelaria                                  155  1.922581    2.0  0.922556
casa_construcao                             66  1.909091    2.0  0.836242
utilidades_domesticas                      586  1.889078    2.0  0.888473
cool_stuff                                 298  1.885906    2.0  0.906618
relogios_presentes                         589  1.881154    2.0  0.905088
pet_shop                                   147  1.857143    2.0  0.883641
moveis_escritorio                          261  1.854406    1.0  0.920843
cama_mesa_banho                           1221  1.850942    2.0  0.890235
esporte_lazer                              626  1.846645    2.0  0.899136
casa_conforto                               57  1.842105    2.0  0.902169
beleza_saude                               716  1.840782    1.0  0.908275
ferramentas_jardim                         318  1.817610    1.0  0.904635
telefonia                                  488  1.815574    2.0  0.875636
moveis_decoracao                           743  1.802153    1.0  0.888620
brinquedos                                 298  1.788591    1.0  0.890808
bebes                                      244  1.786885    1.0  0.877046
eletronicos                                248  1.778226    1.0  0.883502
construcao_ferramentas_construcao           71  1.774648    1.0  0.881511
automotivo                                 370  1.770270    1.0  0.870275
perfumaria                                 235  1.723404    1.0  0.879543
informatica_acessorios                     797  1.708908    1.0  0.867089
eletrodomesticos                            84  1.630952    1.0  0.875087
eletroportateis                             54  1.518519    1.0  0.770708

### (4.2) Worst product categories

👎 Store the five worst categories in terms of *average review score* into a variable called `worst_products`

In [55]:
worst_products = product_categories.tail(73).sort_values(by = [("review_score", "count")],
                                                       ascending = False)
worst_products

review_score                           
                                         count      mean median       std
product_category_name                                                    
cama_mesa_banho                           1221  1.850942    2.0  0.890235
informatica_acessorios                     797  1.708908    1.0  0.867089
moveis_decoracao                           743  1.802153    1.0  0.888620
beleza_saude                               716  1.840782    1.0  0.908275
esporte_lazer                              626  1.846645    2.0  0.899136
relogios_presentes                         589  1.881154    2.0  0.905088
utilidades_domesticas                      586  1.889078    2.0  0.888473
telefonia                                  488  1.815574    2.0  0.875636
automotivo                                 370  1.770270    1.0  0.870275
ferramentas_jardim                         318  1.817610    1.0  0.904635
brinquedos                                 298  1.788591    1.0  0.890808
cool_stuff                                 298  1.885906    2.0  0.906618
moveis_escritorio                          261  1.854406    1.0  0.920843
eletronicos                                248  1.778226    1.0  0.883502
bebes                                      244  1.786885    1.0  0.877046
perfumaria                                 235  1.723404    1.0  0.879543
fashion_bolsas_e_acessorios                155  1.948387    2.0  0.917319
papelaria                                  155  1.922581    2.0  0.922556
pet_shop                                   147  1.857143    2.0  0.883641
consoles_games                              92  1.956522    2.0  0.924787
eletrodomesticos                            84  1.630952    1.0  0.875087
construcao_ferramentas_construcao           71  1.774648    1.0  0.881511
malas_acessorios                            66  1.924242    2.0  0.949666
casa_construcao                             66  1.909091    2.0  0.836242
casa_conforto                               57  1.842105    2.0  0.902169
eletroportateis                             54  1.518519    1.0  0.770708

In [54]:
worst_products = product_categories.tail(5).sort_values(by = [("review_score", "count")],
                                                       ascending = False)
worst_products

review_score                           
                              count      mean median       std
product_category_name                                         
informatica_acessorios          797  1.708908    1.0  0.867089
automotivo                      370  1.770270    1.0  0.870275
perfumaria                      235  1.723404    1.0  0.879543
eletrodomesticos                 84  1.630952    1.0  0.875087
eletroportateis                  54  1.518519    1.0  0.770708

👇 Create a `worst_products_review` DataFrame which contains the `worst_products` only.

In [33]:
worst_products_reviews.info()

NameError: name 'worst_products_reviews' is not defined

In [35]:
worst_products_reviews = df[df.product_category_name.isin(worst_products.index)]
worst_products_reviews[["review_id",
                        "review_score",
                        "product_category_name",
                        "full_review_cleaned",
                        "most_important_topic",
                        "most_important_words"]
      ]

,review_id,review_score,product_category_name,full_review_cleaned,most_important_topic,most_important_words
34,ff722b4c68783a4459a3adb9bb4e1d0d,3,informatica_acessorios,produt cheg pc consegu reconhec port usb,2,"[produt, praz, entreg, nao, vei]"
57,5f938e5f5f2e9a75710b54feeb9ea610,1,eletrodomesticos,medi pec serv,2,"[produt, praz, entreg, nao, vei]"
71,6b341682ab39af9fa00d72d7388c903b,3,automotivo,not compr produt corret cap intern outr,1,"[receb, produt, compr, entreg, receb produt]"
88,b736ff4204044e49e39062584d06fa74,1,eletroportateis,loj anunc produt entreg outr,1,"[receb, produt, compr, entreg, receb produt]"
100,6f885e2fd69412264d9e74e805175d53,1,informatica_acessorios,receb bem rapid porem vei outr produt ped mode...,2,"[produt, praz, entreg, nao, vei]"
...,...,...,...,...,...,...
36049,e9f0bde9a98ff79305964e9033ed8cd2,1,informatica_acessorios,produt cheg falt pess cond,1,"[receb, produt, compr, entreg, receb produt]"
36054,44380245bf1a49875542ea0e7f216986,1,informatica_acessorios,receb produt aind assim receb produt avali posi,0,"[bom, produt, vei, compr, pess]"
36095,970ae7c9df61910a0a30c38670db7ae5,3,informatica_acessorios,probl past multius pra notebook ped cor vermel...,0,"[bom, produt, vei, compr, pess]"
36100,504403288b387de1c95a193909ba5dfd,3,perfumaria,cheg bem rapid produt razoa,2,"[produt, praz, entreg, nao, vei]"


In [36]:
worst_products_reviews.to_csv('worst_products_reviews.csv', index=False)

In [ ]:
worst_products.to_csv('worst_products.csv', index=False)

### (4.3). Topics for the worst products

❓ What are the topics for the worst products ❓

In [37]:
worst_products_reviews["most_important_topic"].value_counts()

most_important_topic
1    528
2    524
0    488
Name: count, dtype: int64

In [38]:
bad_frequency = list(worst_products_reviews["most_important_topic"].value_counts().index)
bad_frequency

[1, 2, 0]

In [39]:
[topic_word_mixture[i] for i in bad_frequency]

[['receb', 'produt', 'compr', 'entreg', 'receb produt'],
 ['produt', 'praz', 'entreg', 'nao', 'vei'],
 ['bom', 'produt', 'vei', 'compr', 'pess']]

## (5) 🎁 Sellers...

* What kind of products were sold by the worst sellers ?
* What are the main reviews for the worst sellers ?

### (5.1) Worst Sellers

In [40]:
sellers = pd.read_csv("sellers_cleaned.csv")
sellers.columns

Index(['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state'], dtype='object')

👇 Select the 10 worst sellers and store them into a variable called `worst_sellers`

In [ ]:
order_items = pd.read_csv('order_item.csv')
df_merged = pd.merge(order_items, sellers, on='seller_id', how='left')
df1 = pd.merge(df, df_merged, on='order_id', how='left')
# Compter les mauvaises reviews (score <= 3) par vendeur
bad_reviews_count = df1[df1['review_score'] <= 3].groupby('seller_id').size().reset_index(name='bad_reviews_count')
bad_reviews_count
df3 = pd.merge(df1, bad_reviews_count, on='seller_id', how='left')


In [41]:
worst_sellers = pd.read_csv('worst_sellers.csv')
worst_sellers

,seller_id,Moyenne_Reviews,Reviews_1_2_3_Stars
0,4a3ca9315b744ce9f8e937436149388,4.780574,667
1,7c67e1448b00f6e969d365cea6b010ab,3.337977,607
2,1025f0e2d44d7041d6cf58b6550e0bfa,3.830532,423
3,955fee9216a65b617aa5c0531780ce60,4.034690,369
4,da8622b14eb17ae2831f4ac5b9dab84a,4.061896,368
5,4869f7a5dfa277a7dca6462dcf3b52b2,4.112457,263
6,7a67c85e85bb2ce8582c35f2203ad736,4.217763,233
7,fa1c13f2614d7b5c4749cbc52fecda94,4.327645,88
8,53243585a1d6dc2643021fd1853d8905,4.068293,88
9,7e93a43ef30c4f03f38b393420bc753a,4.197059,68


### (5.2) Products sold by the worst sellers

In [42]:
order_items = pd.read_csv("order_item.csv")
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112648 entries, 0 to 112647
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112648 non-null  object 
 1   order_item_id        112648 non-null  int64  
 2   product_id           112648 non-null  object 
 3   seller_id            112648 non-null  object 
 4   shipping_limit_date  112648 non-null  object 
 5   price                112648 non-null  float64
 6   freight_value        112648 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [43]:
product = pd.read_csv("products.csv")
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32951 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


❓ What are the types of products sold by the worst sellers ❓

In [44]:
# Si vous avez order_items et products séparés
sellers_product_category = order_items.merge(product, 
                                             on="product_id", 
                                             how="left")[["seller_id", "product_category_name"]]

sellers_product_category

,seller_id,product_category_name
0,48436dade18ac8b2bce089ec2a041202,cool_stuff
1,dd7ddc04e1b6c2c614352b383efe2d36,pet_shop
2,5b51032eddd242adc84c38acab88f23d,furniture_decor
3,9d7a1d34a5052409006425275ba1c2b4,perfumery
4,df560393f3a51e74553ab94004ba5c87,garden_tools
...,...,...
112643,b8bc237ba3788b23da09c0f1f3a3288c,housewares
112644,f3c38ab652836d21de61fb8314b69182,computers_accessories
112645,c3cfdc648177fdbbbb35635a37472c53,sports_leisure
112646,2b3e4a2a3ea8e01938cabda2a3e5cc79,computers_accessories


In [45]:
sellers_product_category.groupby("seller_id").count()

,product_category_name
seller_id,
0015a82c2db000af6aaaf3ae2ecb0532,3
001cca7ae9ae17fb1caed9dfb1094831,239
001e6ad469a905060d959994f1b41e4f,1
002100f778ceb8431b7a1020ff7ab48f,55
003554e2dce176b5555353e4f3555ac8,1
...,...
ffcfefa19b08742c5d315f2791395ee5,1
ffdd9f82b9a447f6f8d4b91554cc7dd3,20
ffeee66ac5d5a62fe688b9d26f83f534,14


### (5.3) Categories and topics for the worst sellers

🎁 Here are some useful functions:
- `focus_seller(seller_id)` to show the product categories sold by a seller
- `bad_reviews_seller` to show to topwords of the most frequent topic for one seller

In [46]:
def focus_seller(seller_id):
    return sellers_product_category[sellers_product_category.seller_id == seller_id].value_counts()

In [52]:
bad_reviews_sellers.to_csv('bad_reviews_sellers.csv', index=False)

In [51]:
# Si vous avez order_items comme dataframe séparé
bad_reviews_sellers = worst_products_reviews.merge(order_items, 
                                                    on="order_id",  # ou la clé commune
                                                    how="left")
bad_reviews_sellers

,review_id,length_review,review_score,order_id,product_category_name,full_review,full_review_cleaned,most_important_topic,most_important_words,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,ff722b4c68783a4459a3adb9bb4e1d0d,66,3,b8702f50a689621ab21ef200b55a5b8d,informatica_acessorios,"Produto chegou, mas meu PC não conseguiu reco...",produt cheg pc consegu reconhec port usb,2,"[produt, praz, entreg, nao, vei]",1,e76438822c17316d276c27e52e588931,5656537e588803a555b8eb41f07a944b,2018-01-10,24.89,17.63
1,5f938e5f5f2e9a75710b54feeb9ea610,17,1,d9ff0185a300435402f043124f530274,eletrodomesticos,Médio A peça não serviu,medi pec serv,2,"[produt, praz, entreg, nao, vei]",1,082175ee534338e4fd967076a6f848e5,d91fb3b7d041e83b64a00a3edfb37e4f,2018-05-10,52.00,18.24
2,6b341682ab39af9fa00d72d7388c903b,66,3,fd65651f855608ad1aa52ff4a741c31a,automotivo,Nota3 Comprar um produto correto na capa mas i...,not compr produt corret cap intern outr,1,"[receb, produt, compr, entreg, receb produt]",1,1dce3fd38c13a3e74fe5000f33858442,72c5da29406b4234927b81855e7b64f6,2018-05-07,229.99,19.49
3,b736ff4204044e49e39062584d06fa74,43,1,993da8482e658ec7f1d0c6d955604936,eletroportateis,A loja anuncia um produto e entrega outro.,loj anunc produt entreg outr,1,"[receb, produt, compr, entreg, receb produt]",1,c5e19a898511a0e8b60bd98fbc1d72eb,2d2c62bef4c8ba0dd74e0e228e808539,2018-08-21,220.00,38.67
4,6f885e2fd69412264d9e74e805175d53,156,1,108e1d5b7a25ff07b49e500b1c7ae4b0,informatica_acessorios,"Recebi bem rápido, porém veio outro produto, ...",receb bem rapid porem vei outr produt ped mode...,2,"[produt, praz, entreg, nao, vei]",1,cb6c1c21ebc9c153a7baf319c515657a,85d9eb9ddc5d00ca9336a2219c97bb13,2017-06-13,31.90,34.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,e9f0bde9a98ff79305964e9033ed8cd2,50,1,0e0abe27977ea1d66568d6ee7d310217,informatica_acessorios,O produto chegou faltando e em péssimas condi...,produt cheg falt pess cond,1,"[receb, produt, compr, entreg, receb produt]",2,c6b3e5dad7e071459c1ab64d517d848c,4e922959ae960d389249c378d1c939f5,2018-02-13,74.00,15.27
2025,44380245bf1a49875542ea0e7f216986,44,1,98efe48952662e6a45a1dc51770039d2,informatica_acessorios,não recebi o produto aind assim que receber pr...,receb produt aind assim receb produt avali posi,0,"[bom, produt, vei, compr, pess]",1,3f14d740544f37ece8a9e7bc8349797e,6973a06f484aacf400ece213dbf3d946,2018-06-22,84.89,13.25
2026,970ae7c9df61910a0a30c38670db7ae5,151,3,b7e332393b5e64cdc10d308c971a41c4,informatica_acessorios,Teve um probleminha minha pasta multiuso pra ...,probl past multius pra notebook ped cor vermel...,0,"[bom, produt, vei, compr, pess]",1,e1a698b55be47318e6d178c57bf5370f,85d9eb9ddc5d00ca9336a2219c97bb13,2018-08-29,38.90,15.37
2027,504403288b387de1c95a193909ba5dfd,39,3,69dc71661dcaa2145e4788d004eb250b,perfumaria,Chegou bem rapidinho. Produto razoável,cheg bem rapid produt razoa,2,"[produt, praz, entreg, nao, vei]",1,9e0e152552a1323f7e5dcf63d50cdae3,cc419e0650a3c5ba77189a1882b7556a,2017-08-03,56.99,15.15


In [48]:
def bad_reviews_seller(bad_reviews_sellers, seller_id):
    mask = (bad_reviews_sellers.seller_id == seller_id)
    temp = bad_reviews_sellers[mask]
    if len(temp) > 0: # if seller appears in the bad reviews dataframe
        most_frequent_topic_seller = list(temp.most_important_topic.value_counts().head(1).index)[0]
        return topic_word_mixture[most_frequent_topic_seller]

❓For each of these worst sellers, show the most frequent product categories and words ❓

In [49]:
for worst_seller in worst_sellers["seller_id"]:
    print("-"*50)
    print(f"Focusing on the seller #{worst_seller}...")
    print(focus_seller(worst_seller))
    print(bad_reviews_seller(bad_reviews_sellers, worst_seller))


--------------------------------------------------
Focusing on the seller #4a3ca9315b744ce9f8e937436149388...
Series([], Name: count, dtype: int64)
None
--------------------------------------------------
Focusing on the seller #7c67e1448b00f6e969d365cea6b010ab...
seller_id                         product_category_name
7c67e1448b00f6e969d365cea6b010ab  office_furniture         1233
                                  furniture_decor            91
                                  sports_leisure             18
                                  furniture_living_room      16
                                  baby                        3
                                  auto                        3
Name: count, dtype: int64
['produt', 'praz', 'entreg', 'nao', 'vei']
--------------------------------------------------
Focusing on the seller #1025f0e2d44d7041d6cf58b6550e0bfa...
seller_id                         product_category_name    
1025f0e2d44d7041d6cf58b6550e0bfa  furniture_decor       

🏁 Congratulations. You've learned some basics of NLP (Preprocessing + Vectorizing + NB/LDA) and we combined this new "expertise" with Decision Science

💾 Don't forget to `git add / commit / push`